## 👉 START HERE: How to use this notebook

### Step 1: Agent storage configuration

This notebook initializes a `AgentStorageConfig` Pydantic class to define the locations where the Agent's code/config and its supporting data & metadata is stored in the Unity Catalog:
- **Unity Catalog Model:** Stores staging/production versions of the Agent's code/config
- **MLflow Experiment:** Stores every development version of the Agent's code/config, each version's associated quality/cost/latency evaluation results, and any MLflow Traces from your development & evaluation processes
- **Evaluation Set Delta Table:** Stores the Agent's evaluation set

This notebook does the following:
1. Validates the provided locations exist.
2. Serializes this configuration to `config/agent_storage_config.yaml` so other notebooks can use it

**Important note:** Throughout this notebook, we indicate which cells you:
- ✅✏️ *should* customize - these cells contain config settings to change
- 🚫✏️ *typically will not* customize - these cells contain boilerplate code required to validate / save the configuration

*Cells that don't require customization still need to be run!*

### 🚫✏️ Install Python libraries

In [0]:
%pip install -qqqq -U -r requirements.txt
%restart_python

### 🚫✏️ Connect to Databricks

If running locally in an IDE using Databricks Connect, connect the Spark client & configure MLflow to use Databricks Managed MLflow.  If this running in a Databricks Notebook, these values are already set.

In [3]:
from mlflow.utils import databricks_utils as du

if not du.is_in_databricks_notebook():
    from databricks.connect import DatabricksSession
    import os

    spark = DatabricksSession.builder.getOrCreate()
    os.environ["MLFLOW_TRACKING_URI"] = "databricks"

### 🚫✏️ Get current user info to set default values

In [4]:
from utils.cookbook.agent_storage_config import get_current_user_info

user_email, user_name, default_catalog = get_current_user_info(spark)

print(f"User email: {user_email}")
print(f"User name: {user_name}")
print(f"Default UC catalog: {default_catalog}")

User email: eric.peter@databricks.com
User name: eric_peter
Default UC catalog: main


### ✅✏️ Configure your Agent's storage locations

Either review & accept the default values or enter your preferred location.

In [11]:
from utils.cookbook.agent_storage_config import AgentStorageConfig
import mlflow

# Default values below for `AgentStorageConfig` 
agent_name = "my_agent"
uc_catalog_name = f"{default_catalog}"
uc_schema_name = f"{user_name}_agents"
uc_catalog_name = f"ep"
uc_schema_name = f"cookbook_local_test"

# Agent storage configuration
agent_storage_config = AgentStorageConfig(
    uc_model_name=f"{uc_catalog_name}.{uc_schema_name}.{agent_name}",  # UC model to store staging/production versions of the Agent's code/config
    evaluation_set_uc_table=f"{uc_catalog_name}.{uc_schema_name}.{agent_name}_eval_set",  # UC table to store the evaluation set
    mlflow_experiment_name=f"/Users/{user_email}/{agent_name}_mlflow_experiment",  # MLflow Experiment to store development versions of the Agent and their associated quality/cost/latency evaluation results + MLflow Traces
)

# Validate the UC catalog and schema for the Agent'smodel & evaluation table
is_valid, msg = agent_storage_config.validate_catalog_and_schema()
if not is_valid:
    raise Exception(msg)

# Set the MLflow experiment, validating the path is valid
mlflow.set_experiment(agent_storage_config.mlflow_experiment_name)
# If running in a local IDE, set the MLflow experiment name as an environment variable
os.environ["MLFLOW_EXPERIMENT_NAME"] = agent_storage_config.mlflow_experiment_name

All catalogs and schemas exist for both model `ep.cookbook_local_test.my_agent` and evaluation table `ep.cookbook_local_test.my_agent_eval_set`.


### 🚫✏️ Save the configuration for use by other notebooks

In [12]:
from utils.agents.tools import obj_to_yaml_file

obj_to_yaml_file(agent_storage_config, "./configs/agent_storage_config.yaml")